In [13]:
import pandas as pd
import json
import numpy as np

In [14]:
df = pd.read_json('purchase_data.json')
df.head()

,Age,Gender,Item ID,Item Name,Price,SN
0,38,Male,165,Bone Crushing Silver Skewer,3.37,Aelalis34
1,21,Male,119,"Stormbringer, Dark Blade of Ending Misery",2.32,Eolo46
2,34,Male,174,Primitive Blade,2.46,Assastnya25
3,21,Male,92,Final Critic,1.36,Pheusrical25
4,23,Male,63,Stormfury Mace,1.27,Aela59


# Total Players

In [15]:
# Calculating Total Players

total_players = len(df["SN"].unique())

print("Total Players {}".format(total_players)) 

Total Players 573


# Purchasing Analysis

In [30]:
# Uniique Items 
unique_items = len(df['Item ID'].unique())

# Average Purchase Price
avg_price = round(df.Price.mean(),2)

# Total Revenue 
total_revenue = "${}".format(round(df.Price.sum(),2))

# Total Number of Purchases
total_purchases = df.shape[0]

purchasing_analysis = pd.DataFrame({"Number of Unique Items": [unique_items],
                                    "Average Purchase Price": [avg_price],
                                    "Number of Purchases": [total_purchases],
                                    "Total Revenue": [total_revenue]})
purchasing_analysis

,Average Purchase Price,Number of Purchases,Number of Unique Items,Total Revenue
0,2.93,780,183,$2286.33


# Gender Demographics

In [45]:
# Gender Demographics: % of Players and Total Count
normed = df.groupby(['Gender', 'SN']).count().reset_index()['Gender'].value_counts(normalize=True)
absolute = df.groupby(['Gender', 'SN']).count().reset_index()['Gender'].value_counts(normalize=False)

pd.concat([normed, absolute], axis=1)

,Gender,Gender
Male,0.811518,465
Female,0.174520,100
Other / Non-Disclosed,0.013962,8


# Purchasing Analysis (Gender)

In [57]:
# Analysis by Gender

purchases_gender = df.groupby('Gender').agg(['sum', 'mean', 'count'])

level_0 = purchases_gender.columns.get_level_values(0)
level_1 = purchases_gender.columns.get_level_values(1)
purchases_gender.columns = level_0 + '_' + level_1
purchases_gender = purchases_gender[['Price_sum', 'Price_mean', 'Price_count',]]

purchases_gender = pd.concat([purchases_gender, absolute], axis=1)
purchases_gender['Normalized'] = purchases_gender.Price_sum / purchases_gender.Gender
purchases_gender

,Price_sum,Price_mean,Price_count,Gender,Normalized
Female,382.91,2.815515,136,100,3.829100
Male,1867.68,2.950521,633,465,4.016516
Other / Non-Disclosed,35.74,3.249091,11,8,4.467500


# Age Demographics

In [52]:
# Bins for age groups

bins = [0, 10, 14, 19, 24, 29, 34, 39, 100]
age_labels = ['<10', '10-14', '15-19', '20-24', '25-29', '30-34', '35-39', '40+']

df['Age Groups'] =pd.cut(df["Age"], bins, labels=age_labels)
age_count = (df.groupby("Age Groups"))['SN'].nunique().reset_index()
age_count = age_count.rename(columns={'SN': 'Total Count'})
age_count


,Age Groups,Total Count
0,<10,22
1,10-14,20
2,15-19,100
3,20-24,259
4,25-29,87
5,30-34,47
6,35-39,27
7,40+,11


In [99]:
age_norm = df.groupby('Age Groups').agg(['sum', 'mean', 'count'])['Price']
age_norm.reset_index(inplace=True)

age_norm

,Age Groups,sum,mean,count
0,<10,96.62,3.019375,32
1,10-14,83.79,2.702903,31
2,15-19,386.42,2.905414,133
3,20-24,978.77,2.913006,336
4,25-29,370.33,2.962640,125
5,30-34,197.25,3.082031,64
6,35-39,119.40,2.842857,42
7,40+,53.75,3.161765,17


# Top Spenders

In [98]:
#Top Spenders by purchase value

df_top = df.groupby('SN')['Price'].agg(['sum', 'mean', 'count']).sort_values(by='sum', ascending=False).head()
df_top = df_top.rename(columns={'sum': 'Total Purchase Value', 'mean': 'Average Purchase Price', 'count': 'Purchase Count'})
df_top['Total Purchase Value'] = df_top['Total Purchase Value'].map('${:.2f}'.format)
df_top['Average Purchase Price'] = df_top['Average Purchase Price'].map('${:.2f}'.format)
df_top

,Total Purchase Value,Average Purchase Price,Purchase Count
SN,,,
Undirrala66,$17.06,$3.41,5
Saedue76,$13.56,$3.39,4
Mindimnya67,$12.74,$3.18,4
Haellysu29,$12.73,$4.24,3
Eoda93,$11.58,$3.86,3


# Most Popular

In [93]:
# Most popular items by number of purchase

df_popular = df.groupby('Item Name')['Price'].agg(['sum', 'mean', 'count']).sort_values(by='count', ascending=False).head()
df_popular = df_popular.rename(columns={'sum': 'Total Purchase Price', 'mean': 'Average Price', 'count': 'Purchase Count'})
df_popular['Total Purchase Price'] = df_popular['Total Purchase Price'].map('${:.2f}'.format)
df_popular['Average Price'] = df_popular['Average Price'].map('${:.2f}'.format)
df_popular


,Total Purchase Price,Average Price,Purchase Count
Item Name,,,
Final Critic,$38.60,$2.76,14
Arcane Gem,$24.53,$2.23,11
"Betrayal, Whisper of Grieving Widows",$25.85,$2.35,11
Stormcaller,$34.65,$3.46,10
Woeful Adamantite Claymore,$11.16,$1.24,9


# Most Profitable

In [87]:
# Top Items by purchase value

df_profit = df.groupby(['Item Name', 'Item ID', 'Price']).Price.agg(['count', 'mean', 'sum']).head().reset_index()
df_profit = df_profit.rename(columns={'count': 'Purchase Count', 'mean': 'Average Price','sum': 'Total Purchase Value'})
df_profit['Total Purchase Value'] = df_profit['Total Purchase Value'].map('${:.2f}'.format)
df_profit['Price'] = df_profit['Price'].map('${:.2f}'.format)
del df_profit['Average Price']
df_profit

,Item Name,Item ID,Price,Purchase Count,Total Purchase Value
0,Abyssal Shard,162,$2.04,3,$6.12
1,"Aetherius, Boon of the Blessed",137,$4.75,4,$19.00
2,Agatha,120,$1.91,5,$9.55
3,Alpha,130,$1.56,7,$10.92
4,"Alpha, Oath of Zeal",79,$2.88,7,$20.16


# Conclusions

In [ ]:
# 1. The majority of players are male.
# 2. 20-24 Year old customers are making the most purchases and spending the most money.
# 3. The sales for the top 5 items didn't vary much.